In [5]:
# look at tools/set_up_magics.ipynb
yandex_metrica_allowed = True ; get_ipython().run_cell('# one_liner_str\n\nget_ipython().run_cell_magic(\'javascript\', \'\', \n    \'// setup cpp code highlighting\\n\'\n    \'IPython.CodeCell.options_default.highlight_modes["text/x-c++src"] = {\\\'reg\\\':[/^%%cpp/]} ;\'\n    \'IPython.CodeCell.options_default.highlight_modes["text/x-cmake"] = {\\\'reg\\\':[/^%%cmake/]} ;\'\n)\n\n# creating magics\nfrom IPython.core.magic import register_cell_magic, register_line_magic\nfrom IPython.display import display, Markdown, HTML\nimport argparse\nfrom subprocess import Popen, PIPE\nimport random\nimport sys\nimport os\nimport re\nimport signal\nimport shutil\nimport shlex\nimport glob\nimport time\n\n@register_cell_magic\ndef save_file(args_str, cell, line_comment_start="#"):\n    parser = argparse.ArgumentParser()\n    parser.add_argument("fname")\n    parser.add_argument("--ejudge-style", action="store_true")\n    args = parser.parse_args(args_str.split())\n    \n    cell = cell if cell[-1] == \'\\n\' or args.no_eof_newline else cell + "\\n"\n    cmds = []\n    with open(args.fname, "w") as f:\n        f.write(line_comment_start + " %%cpp " + args_str + "\\n")\n        for line in cell.split("\\n"):\n            line_to_write = (line if not args.ejudge_style else line.rstrip()) + "\\n"\n            if line.startswith("%"):\n                run_prefix = "%run "\n                if line.startswith(run_prefix):\n                    cmds.append(line[len(run_prefix):].strip())\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                comment_prefix = "%" + line_comment_start\n                if line.startswith(comment_prefix):\n                    cmds.append(\'#\' + line[len(comment_prefix):].strip())\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                raise Exception("Unknown %%save_file subcommand: \'%s\'" % line)\n            else:\n                f.write(line_to_write)\n        f.write("" if not args.ejudge_style else line_comment_start + r" line without \\n")\n    for cmd in cmds:\n        if cmd.startswith(\'#\'):\n            display(Markdown("\\#\\#\\#\\# `%s`" % cmd[1:]))\n        else:\n            display(Markdown("Run: `%s`" % cmd))\n            get_ipython().system(cmd)\n\n@register_cell_magic\ndef cpp(fname, cell):\n    save_file(fname, cell, "//")\n    \n@register_cell_magic\ndef cmake(fname, cell):\n    save_file(fname, cell, "#")\n\n@register_cell_magic\ndef asm(fname, cell):\n    save_file(fname, cell, "//")\n    \n@register_cell_magic\ndef makefile(fname, cell):\n    fname = fname or "makefile"\n    assert fname.endswith("makefile")\n    save_file(fname, cell.replace(" " * 4, "\\t"))\n        \n@register_line_magic\ndef p(line):\n    line = line.strip() \n    if line[0] == \'#\':\n        display(Markdown(line[1:].strip()))\n    else:\n        try:\n            expr, comment = line.split(" #")\n            display(Markdown("`{} = {}`  # {}".format(expr.strip(), eval(expr), comment.strip())))\n        except:\n            display(Markdown("{} = {}".format(line, eval(line))))\n    \n    \ndef show_log_file(file, return_html_string=False):\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function halt__OBJ__(elem, color)\n        {\n            elem.setAttribute("style", "font-size: 14px; background: " + color + "; padding: 10px; border: 3px; border-radius: 5px; color: white; ");                    \n        }\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            if (entrance___OBJ__ < 0) {\n                entrance___OBJ__ = 0;\n            }\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                if (elem.innerHTML != xmlhttp.responseText) {\n                                    elem.innerHTML = xmlhttp.responseText;\n                                }\n                                if (elem.innerHTML.includes("Process finished.")) {\n                                    halt__OBJ__(elem, "#333333");\n                                } else {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                }\n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (!entrance___OBJ__) {\n                                if (errors___OBJ__ < 6) {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                } else {\n                                    halt__OBJ__(elem, "#994444");\n                                }\n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n\n        <p id="__OBJ__" style="font-size: 14px; background: #000000; padding: 10px; border: 3px; border-radius: 5px; color: white; ">\n        </p>\n        \n        </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__.md -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n\n    \nclass TInteractiveLauncher:\n    tmp_path = "./interactive_launcher_tmp"\n    def __init__(self, cmd):\n        try:\n            os.mkdir(TInteractiveLauncher.tmp_path)\n        except:\n            pass\n        name = str(random.randint(0, 1e18))\n        self.inq_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".inq")\n        self.log_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".log")\n        \n        os.mkfifo(self.inq_path)\n        open(self.log_path, \'w\').close()\n        open(self.log_path + ".md", \'w\').close()\n\n        self.pid = os.fork()\n        if self.pid == -1:\n            print("Error")\n        if self.pid == 0:\n            exe_cands = glob.glob("../tools/launcher.py") + glob.glob("../../tools/launcher.py")\n            assert(len(exe_cands) == 1)\n            assert(os.execvp("python3", ["python3", exe_cands[0], "-l", self.log_path, "-i", self.inq_path, "-c", cmd]) == 0)\n        self.inq_f = open(self.inq_path, "w")\n        interactive_launcher_opened_set.add(self.pid)\n        show_log_file(self.log_path)\n\n    def write(self, s):\n        s = s.encode()\n        assert len(s) == os.write(self.inq_f.fileno(), s)\n        \n    def get_pid(self):\n        n = 100\n        for i in range(n):\n            try:\n                return int(re.findall(r"PID = (\\d+)", open(self.log_path).readline())[0])\n            except:\n                if i + 1 == n:\n                    raise\n                time.sleep(0.1)\n        \n    def input_queue_path(self):\n        return self.inq_path\n        \n    def wait_stop(self, timeout):\n        for i in range(int(timeout * 10)):\n            wpid, status = os.waitpid(self.pid, os.WNOHANG)\n            if wpid != 0:\n                return True\n            time.sleep(0.1)\n        return False\n        \n    def close(self, timeout=3):\n        self.inq_f.close()\n        if not self.wait_stop(timeout):\n            os.kill(self.get_pid(), signal.SIGKILL)\n            os.waitpid(self.pid, 0)\n        os.remove(self.inq_path)\n        # os.remove(self.log_path)\n        self.inq_path = None\n        self.log_path = None \n        interactive_launcher_opened_set.remove(self.pid)\n        self.pid = None\n        \n    @staticmethod\n    def terminate_all():\n        if "interactive_launcher_opened_set" not in globals():\n            globals()["interactive_launcher_opened_set"] = set()\n        global interactive_launcher_opened_set\n        for pid in interactive_launcher_opened_set:\n            print("Terminate pid=" + str(pid), file=sys.stderr)\n            os.kill(pid, signal.SIGKILL)\n            os.waitpid(pid, 0)\n        interactive_launcher_opened_set = set()\n        if os.path.exists(TInteractiveLauncher.tmp_path):\n            shutil.rmtree(TInteractiveLauncher.tmp_path)\n    \nTInteractiveLauncher.terminate_all()\n   \nyandex_metrica_allowed = bool(globals().get("yandex_metrica_allowed", False))\nif yandex_metrica_allowed:\n    display(HTML(\'\'\'<!-- YANDEX_METRICA_BEGIN -->\n    <script type="text/javascript" >\n       (function(m,e,t,r,i,k,a){m[i]=m[i]||function(){(m[i].a=m[i].a||[]).push(arguments)};\n       m[i].l=1*new Date();k=e.createElement(t),a=e.getElementsByTagName(t)[0],k.async=1,k.src=r,a.parentNode.insertBefore(k,a)})\n       (window, document, "script", "https://mc.yandex.ru/metrika/tag.js", "ym");\n\n       ym(59260609, "init", {\n            clickmap:true,\n            trackLinks:true,\n            accurateTrackBounce:true\n       });\n    </script>\n    <noscript><div><img src="https://mc.yandex.ru/watch/59260609" style="position:absolute; left:-9999px;" alt="" /></div></noscript>\n    <!-- YANDEX_METRICA_END -->\'\'\'))\n\ndef make_oneliner():\n    html_text = \'("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "")\'\n    html_text += \' + "<""!-- MAGICS_SETUP_PRINTING_END -->"\'\n    return \'\'.join([\n        \'# look at tools/set_up_magics.ipynb\\n\',\n        \'yandex_metrica_allowed = True ; get_ipython().run_cell(%s);\' % repr(one_liner_str),\n        \'display(HTML(%s))\' % html_text,\n        \' #\'\'MAGICS_SETUP_END\'\n    ])\n       \n\n');display(HTML(("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "") + "<""!-- MAGICS_SETUP_PRINTING_END -->")) #MAGICS_SETUP_END

<IPython.core.display.Javascript object>

# Assembler x86-64

<table width=100%> <tr>
    <th width=20%> <b>Видеозапись семинара &rarr; </b> </th>
    <th>
    <a href="https://www.youtube.com/watch?v=i_eeouEiXnI&list=PLjzMm8llUm4AmU6i_hPU0NobgA4VsBowc&index=8">
        <img src="video.jpg" width="320"  height="160" align="left" alt="Видео с семинара"> 
    </a>
    </th>
    <th> </th>
</tr> </table>

## Особенности
* Много регистров
* Много команд
* Много легаси
* Разные синтаксисы


[Ридинг Яковлева](https://github.com/victor-yacovlev/mipt-diht-caos/tree/master/practice/asm/x86_basics) 


Сегодня в программе:
* <a href="#regs" style="color:#856024"> Регистры </a>
* <a href="#syntax" style="color:#856024"> Синтаксисы </a>
* <a href="#clamp" style="color:#856024"> Функция clamp </a>
* <a href="#mem" style="color:#856024"> Работа с памятью </a>
* <a href="#call" style="color:#856024"> Вызов функций </a>
* <a href="#mul" style="color:#856024"> Интересные факты </a>
* <a href="#hw" style="color:#856024"> Комментарии к ДЗ </a>

#  <a name="regs"></a> Регистры

Немного истории

| Год  | Регистры           | Битность | Первый процессор | Комментарий |
|------|--------------------|----------|------------------|-------------|
| 1974 | a, b, c, d         | 8 bit    | Intel 8080       | |
| 1978 | ax, bx, cx, dx     | 16 bit   | Intel 8086       | X - eXtended ([совсем ненадежный источник](https://stackoverflow.com/a/892948))|
| 1985 | eax, ebx, exc, edx | 32 bit   | Intel 80386      | E - extended |
| 2003 | rax, rbx, rcx, rdx | 64 bit   | AMD Opteron      | R - (внезапно) register |


Как оно выглядит сейчас в 64-битных процессорах

<table width="800px" border="1" style="text-align:center; font-family: Courier New; font-size: 10pt">

<tbody><tr>
<td colspan="8" width="25%" style="background:lightgrey">RAX
</td>
<td colspan="8" width="25%" style="background:lightgrey">RCX
</td>
<td colspan="8" width="25%" style="background:lightgrey">RDX
</td>
<td colspan="8" width="25%" style="background:lightgrey">RBX
</td></tr>
<tr>
<td colspan="4" width="12.5%"></td>
<td colspan="4" width="12.5%" style="background:lightgrey">EAX
</td>
<td colspan="4" width="12.5%"></td>
<td colspan="4" width="12.5%" style="background:lightgrey">ECX
</td>
<td colspan="4" width="12.5%"></td>
<td colspan="4" width="12.5%" style="background:lightgrey">EDX
</td>
<td colspan="4" width="12.5%"></td>
<td colspan="4" width="12.5%" style="background:lightgrey">EBX
</td></tr>
<tr>
<td colspan="6" width="18.75%"></td>
<td colspan="2" width="6.25%" style="background:lightgrey">AX
</td>
<td colspan="6" width="18.75%"></td>
<td colspan="2" width="6.25%" style="background:lightgrey">CX
</td>
<td colspan="6" width="18.75%"></td>
<td colspan="2" width="6.25%" style="background:lightgrey">DX
</td>
<td colspan="6" width="18.75%"></td>
<td colspan="2" width="6.25%" style="background:lightgrey">BX
</td></tr>
<tr>
<td colspan="6" width="18.75%"></td>
<td width="3.125%" style="background:lightgrey">AH</td>
<td width="3.125%" style="background:lightgrey">AL
</td>
<td colspan="6" width="18.75%"></td>
<td width="3.125%" style="background:lightgrey">CH</td>
<td width="3.125%" style="background:lightgrey">CL
</td>
<td colspan="6" width="18.75%"></td>
<td width="3.125%" style="background:lightgrey">DH</td>
<td width="3.125%" style="background:lightgrey">DL
</td>
<td colspan="6" width="18.75%"></td>
<td width="3.125%" style="background:lightgrey">BH</td>
<td width="3.125%" style="background:lightgrey">BL
</td></tr></tbody></table>

(На самом деле все далеко не так просто устроено. [stackoverflow](https://stackoverflow.com/a/25456097))

Регистры x86 и их странные названия
* RAX - Accumulator Register
* RBX - Base Register
* RCX - Counter Register
* RDX - Data Register
* RSI - Source Index
* RDI - Destination Index
* RBP - Base Pointer
* RSP - Stack Pointer
* R8...R15 - дополнительные регистры общего назначения

Регистры в x86-64:
    
<br> `rax`, `rbx`, `rcx`, `rdx` - регистры общего назначения.
<br> `rsp` - указатель на вершину стека
<br> `rbp` - указатель на начало фрейма (но можно использовать аккуратно использовать как регистр общего назначения)
<br> `rsi`, `rdi` - странные регистры для копирования массива, по сути регистры общего назначения, но ограниченные в возможностях.
<br> `r8`...`r15`

Целочисленные аргументы передаются последовательно в регистрах: `rdi`, `rsi`, `rdx`, `rcx`, `r8`, `r9`. Если передается более 6 аргументов, то оставшиеся - через стек.
Вещественные аргументы передаются через регистры `xmm0`...`xmm7`.

Возвращаемое значение записывается в регистр `rax`.

Вызываемая функция **обязана сохранять на стеке значения регистров** общего назначения `rbx`, `rbp`, `r12`...`r15`. ([Почему `rbx` в этом ряду?](https://stackoverflow.com/questions/22214208/x86-assembly-why-is-ebx-preserved-in-calling-conventions))

Кроме того, при вызове функции для 64-разрядной архитектуры есть дополнительное требование - перед вызовом функции стек должен быть выровнен по границе 16 байт, то есть необходимо уменьшить значение rsp таким образом, оно было кратно 16. Если кроме регистров задействуется стек для передачи параметров, то они должны быть прижаты к нижней выровненной границе стека.

Для функций гарантируется 128-байтная "красная зона" в стеке ниже регистра rsp - область, которая не будет затронута внешним событием, например, обработчиком сигнала. Таким образом, можно задействовать для адресации локальных переменных память до rsp-128.

In [76]:
%%save_file asm_filter_useless
%run chmod +x asm_filter_useless
#!/bin/bash
grep -v "^\s*\." | grep -v "^[0-9]"

Run: `chmod +x asm_filter_useless`

#  <a name="syntax"></a> Syntaxes
### AT&T

In [23]:
%%cpp att_example.c
%run gcc -m64 -masm=att -O3 att_example.c -S -o att_example.S
%run cat att_example.S | ./asm_filter_useless

#include <stdint.h>

//  rdi, rsi, rdx, rcx, r8, r9
int64_t sum(int32_t a, int32_t b, int32_t c, int32_t d, int32_t e, int32_t f, int32_t g, int64_t h) {
    return a + b + c + d + e + f + g + h;
}

Run: `gcc -fcf-protection=none -m64 -masm=att -O3 att_example.c -S -o att_example.S`

Run: `cat att_example.S | ./asm_filter_useless`

sum:
	addl	%esi, %edi
	movl	8(%rsp), %eax
	addl	%edx, %edi
	addl	%ecx, %edi
	addl	%r8d, %edi
	addl	%r9d, %edi
	addl	%edi, %eax
	cltq
	addq	16(%rsp), %rax
	ret


### Intel

QWORD PTR — это переменная типа четверного слова, DWORD PTR — это переменная типа двойного слова. Слово — это 16 бит. Термин получил распространение в эпоху 16-ти битных процессоров, тогда в регистр помещалось ровно 16 бит. Такой объем информации стали называть словом (word). Т. е. в нашем случае dword (double word) 2*16 = 32 бита = 4 байта (обычный int). 

https://habr.com/ru/post/344896/

In [77]:
%%cpp intel_example.c
%run gcc -m64 -masm=intel -O3 intel_example.c -S -o intel_example.S
%run cat intel_example.S | ./asm_filter_useless

#include <stdint.h>
    
//  rdi, rsi, rdx, rcx, r8, r9
int64_t sum(int32_t a, int32_t b, int32_t c, int32_t d, int32_t e, int32_t f, int32_t g, int64_t h) {
    return a + b + c + d + e + f + g +  h;
}

Run: `gcc -m64 -masm=intel -O3 intel_example.c -S -o intel_example.S`

Run: `cat intel_example.S | ./asm_filter_useless`

sum:
	endbr64
	add	edi, esi
	mov	eax, DWORD PTR 8[rsp]
	add	edi, edx
	add	edi, ecx
	add	edi, r8d
	add	edi, r9d
	add	eax, edi
	cdqe
	add	rax, QWORD PTR 16[rsp]
	ret


Про `endbr64` [Введение в аппаратную защиту стека / Хабр](https://habr.com/ru/post/494000/) и [control-flow-enforcement-technology](https://software.intel.com/sites/default/files/managed/4d/2a/control-flow-enforcement-technology-preview.pdf)

Можно отключить опцией `-fcf-protection=none`. 

Про `cdqe` = `cltq` (это синонимы) - расширяет знаковое 32-битное до 64-битного знакового числа.

TLDR: чтобы хакерам было сложнее, есть особый режим процессора, в котором переход (jump) к инструкции не являющейся `endbr*` приводит к прерыванию и завершению программы.

#  <a name="clamp"></a> Пишем функцию clamp тремя способами

In [22]:
%%asm clamp_disasm.S
.intel_syntax noprefix
.text
.globl clamp    
clamp:
    endbr64
    // x - edi
    // a - esi
    // b - edx
    mov eax, esi // eax = a
    cmp edi, esi // x ? a
    jl .clamp_return // if (x < a) goto .clamp_return;
    cmp edi, edx // x ? b
    mov eax, edx // eax = b
    cmovle eax, edi // if (x <= b) eax = x;
.clamp_return:
    ret // return eax;

In [23]:
%%asm clamp_if.S
.intel_syntax noprefix
.text
.globl clamp
clamp:
    endbr64
    // x - edi
    // a - esi
    // b - edx
    mov eax, esi // eax = a
    cmp edi, esi // x ? a
    jl .clamp_return // if (x < a) goto .clamp_return;
    mov eax, edx
    cmp edi, edx // x ? b
    jg .clamp_return // if (x > b) goto .clamp_return;
    mov eax, edi // eax = x
.clamp_return:
    ret // return eax;

In [24]:
%%asm clamp_cmov.S
.intel_syntax noprefix
.text
.globl clamp    
clamp:
    endbr64
    // x - edi
    // a - esi
    // b - edx
    mov eax, edi // eax = x
    cmp eax, esi // x ? a
    cmovl eax, esi // if (x < a) eax = a
    cmp eax, edx // x ? b
    cmovg eax, edx // if (x > b) eax = b;
    ret // return eax;

In [83]:
%%cpp clamp.c
%run # gcc -S -m64 -masm=intel -O2 clamp.c -o /dev/stdout
#include <stdint.h>

int32_t clamp(int32_t x, int32_t a, int32_t b) {
    if (x < a) return a;
    if (x > b) return b;
    return x;
}

\#\#\#\# ` gcc -S -m64 -masm=intel -O2 clamp.c -o /dev/stdout`

In [80]:
%%cpp clamp_test.c
// compile and test using all three asm clamp implementations
%run gcc -m64 -masm=intel -O2 clamp_disasm.S clamp_test.c -o clamp_test.exe
%run ./clamp_test.exe
%run gcc -m64 -masm=intel -O2 clamp_if.S clamp_test.c -o clamp_if_test.exe
%run ./clamp_if_test.exe
%run gcc -m64 -masm=intel -O2 clamp_cmov.S clamp_test.c -o clamp_cmov_test.exe
%run ./clamp_cmov_test.exe
%run gcc -m64 -masm=intel -O2 clamp.c clamp_test.c -o clamp_cmov_c_test.exe
%run ./clamp_cmov_c_test.exe

#include <stdint.h>
#include <stdio.h>
#include <assert.h>
    

int32_t clamp(int32_t x, int32_t a, int32_t b);

int main() {
    assert(clamp(1, 10, 20) == 10);
    assert(clamp(100, 10, 20) == 20);
    assert(clamp(15, 10, 20) == 15);
    fprintf(stderr, "All is OK");
    return 0;
}

Run: `gcc -m64 -masm=intel -O2 clamp_disasm.S clamp_test.c -o clamp_test.exe`

Run: `./clamp_test.exe`

All is OK

Run: `gcc -m64 -masm=intel -O2 clamp_if.S clamp_test.c -o clamp_if_test.exe`

Run: `./clamp_if_test.exe`

...

Run: `./clamp_cmov_test.exe`

All is OK

Run: `gcc -m64 -masm=intel -O2 clamp.c clamp_test.c -o clamp_cmov_c_test.exe`

Run: `./clamp_cmov_c_test.exe`

All is OK

То же самое ассемблерной вставкой

In [81]:
%%cpp clamp_inline_test.c
%run gcc -m64 -masm=intel -O2 clamp_inline_test.c -o clamp_inline_test.exe
%run ./clamp_inline_test.exe

#include <stdint.h>
#include <stdio.h>
#include <assert.h>
    
int32_t clamp(int32_t a, int32_t b, int32_t c);
__asm__(R"(
clamp:
    endbr64
    mov eax, esi
    cmp edi, esi
    jl .clamp_return
    cmp edi, edx
    mov eax, edx
    cmovle eax, edi
.clamp_return:
    ret
)");

int main() {
    assert(clamp(1, 10, 20) == 10);
    assert(clamp(100, 10, 20) == 20);
    assert(clamp(15, 10, 20) == 15);
    fprintf(stderr, "All is OK");
    return 0;
}

Run: `gcc -m64 -masm=intel -O2 clamp_inline_test.c -o clamp_inline_test.exe`

Run: `./clamp_inline_test.exe`

All is OK

#  <a name="mem"></a> Поработаем с памятью

Даны n, x. Посчитаем $\sum_{i=0}^{n - 1} (-1)^i \cdot x[i]$

In [66]:
%%asm my_sum.S
.intel_syntax noprefix
.text
.globl my_sum
my_sum:
    // n - edi
    // x - rsi
    mov eax, 0
    cmp edi, 0
start_loop:   
    jle return_eax
    add eax, DWORD PTR [rsi]
    add rsi, 4
    dec edi // and write compare with 0 flags
    
    jle return_eax
    sub eax, DWORD PTR [rsi]
    add rsi, 4
    dec edi // and write compare with 0 flags
    
    jmp start_loop
return_eax:
    ret

In [65]:
%%cpp my_sum_test.c
%run gcc -g3 -m64 -masm=intel my_sum_test.c my_sum.S -o my_sum_test.exe
%run ./my_sum_test.exe

#include <stdint.h>
#include <stdio.h>
#include <assert.h>
    
int32_t my_sum(int32_t n, int32_t* x);

int main() {
    int32_t x[] = {100, 2, 200, 3};
    assert(my_sum(sizeof(x) / sizeof(int32_t), x) == 100 - 2 + 200 - 3);
    int32_t y[] = {100, 2, 200};
    assert(my_sum(sizeof(y) / sizeof(int32_t), y) == 100 - 2 + 200);
    int32_t z[] = {100};
    assert(my_sum(sizeof(z) / sizeof(int32_t), z) == 100);
    printf("SUCCESS");
    return 0;
}

Run: `gcc -g3 -m64 -masm=intel my_sum_test.c my_sum.S -o my_sum_test.exe`

Run: `./my_sum_test.exe`

SUCCESS


# <a name="call"></a> Вызов функций

In [5]:
%%cpp example.c
%run gcc -m64 -masm=intel -Os example.c -S -o example.S
%run cat example.S 

#include <stdio.h>
    
int hello(int a, int b) {
    printf("Hello %d and %d\n", a, b);
    return a;
}

Run: `gcc -m64 -masm=intel -Os example.c -S -o example.S`

Run: `cat example.S`

	.file	"example.c"
	.intel_syntax noprefix
	.text
	.section	.rodata.str1.1,"aMS",@progbits,1
.LC0:
	.string	"Hello %d and %d\n"
	.text
	.globl	hello
	.type	hello, @function
hello:
.LFB13:
	.cfi_startproc
	endbr64
	push	r12
	.cfi_def_cfa_offset 16
	.cfi_offset 12, -16
	mov	ecx, esi
	mov	r12d, edi
	mov	edx, edi
	lea	rsi, .LC0[rip]
	mov	edi, 1
	xor	eax, eax
	call	__printf_chk@PLT
	mov	eax, r12d
	pop	r12
	.cfi_def_cfa_offset 8
	ret
	.cfi_endproc
.LFE13:
	.size	hello, .-hello
	.ident	"GCC: (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0"
	.section	.note.GNU-stack,"",@progbits
	.section	.note.gnu.property,"a"
	.align 8
	.long	 1f - 0f
	.long	 4f - 1f
	.long	 5
0:
	.string	 "GNU"
1:
	.align 8
	.long	 0xc0000002
	.long	 3f - 2f
2:
	.long	 0x3
3:
	.align 8
4:


In [9]:
%%cpp print.c
%run gcc -m64 -O3 print.c -o print.exe
%run ./print.exe

#include <stdio.h>
#include <stdint.h>
#include <assert.h>
    
// int hello(int a, int b) {
//     printf("Hello %d and %d\n", a, b);
//     return a;
// }
    
int hello(int a, int b);
__asm__(R"(
.intel_syntax noprefix

.format_s:
   .string "Hello %d and %d\n"
hello:
    /* rdi, rsi, rdx, rcx */
    push rdi /* сохраняем a и заодно обеспечиваем выравнивание по 16 байт для вызываемой функции */
    /* готовим аргументы для printf */
    mov rdx, rsi
    mov rsi, rdi
    lea rdi, .format_s[rip] /* вычисляем адрес форматной строки (он в данном случае относительный, а не абсолютный) */
    call printf@PLT /* вызываем функцию */
    pop rax /* восстанавливаем a и готовимся его возвращать */
    ret
    
.att_syntax prefix /* thanks to Askhat Khayrullin */
)");


int main() {
    hello(1, 2);
    hello(10, 20);
    printf("SUCCESS\n");
}

Run: `gcc -m64 -O3 print.c -o print.exe`

Run: `./print.exe`

Hello 1 and 2
Hello 10 and 20
SUCCESS


In [20]:
!objdump -M intel -d print.exe | grep "<hello>:" -A 10

00000000000011ba <hello>:
    11ba:	57                   	push   rdi
    11bb:	48 89 f2             	mov    rdx,rsi
    11be:	48 89 fe             	mov    rsi,rdi
    11c1:	48 8d 3d e1 ff ff ff 	lea    rdi,[rip+0xffffffffffffffe1]        # 11a9 <.format_s>
    11c8:	e8 a3 fe ff ff       	call   1070 <printf@plt>
    11cd:	58                   	pop    rax
    11ce:	c3                   	ret    
    11cf:	90                   	nop

00000000000011d0 <__libc_csu_init>:


#  <a name="mul"></a> Развлекательно-познавательная часть


In [112]:
%%cpp mul.c
%run gcc -m64 -masm=intel -O1 mul.c -S -o mul.S
%run cat mul.S | ./asm_filter_useless

#include <stdint.h>
    
int32_t mul(int32_t a) { 
    return a * 5;
}

Run: `gcc -m64 -masm=intel -O1 mul.c -S -o mul.S`

Run: `cat mul.S | ./asm_filter_useless`

mul:
	endbr64
	lea	eax, [rdi+rdi*4]
	ret


In [109]:
%%cpp div_0.c
%run gcc -m64 -masm=intel -O3 div_0.c -S -o div_0.S
%run cat div_0.S | ./asm_filter_useless

#include <stdint.h>
    
uint32_t div(uint32_t a) { 
    return a / 12312423;
}

uint32_t div2(uint32_t a, uint32_t b) { 
    return a / b;
}

Run: `gcc -m64 -masm=intel -O3 div_0.c -S -o div_0.S`

Run: `cat div_0.S | ./asm_filter_useless`

div:
	endbr64
	mov	eax, edi
	imul	rax, rax, 1557462729
	shr	rax, 32
	sub	edi, eax
	shr	edi
	add	eax, edi
	shr	eax, 23
	ret
div2:
	endbr64
	mov	eax, edi
	xor	edx, edx
	div	esi
	ret


In [110]:
%%cpp div.c
%run gcc -m64 -masm=intel -O3 div.c -S -o div.S
%run cat div.S | ./asm_filter_useless

#include <stdint.h>
    
int32_t div(int32_t a) { 
    return a / 4;
}

uint32_t udiv(uint32_t a) { 
    return a / 4;
}

Run: `gcc -m64 -masm=intel -O3 div.c -S -o div.S`

Run: `cat div.S | ./asm_filter_useless`

div:
	endbr64
	test	edi, edi
	lea	eax, 3[rdi]
	cmovns	eax, edi
	sar	eax, 2
	ret
udiv:
	endbr64
	mov	eax, edi
	shr	eax, 2
	ret


```
-1: 1111 -> 1111
-2: 1110 -> 1111
-3: 1101 -> 1111
-4: 1100 -> 1111
-5: 1011 -> 1110
```

```
-1: 1111 -> 0010 -> 0
-2: 1110 -> 0001 -> 0
-3: 1101 -> 0000 -> 0
-4: 1100 -> 1111 -> -1
-5: 1011 -> 1110 -> -1
```


# <a name="inline"></a> Inline ASM
http://asm.sourceforge.net/articles/linasm.html

In [74]:
%%cpp simdiv.c
%run gcc -m64 -masm=intel -O3 simdiv.c -o simdiv.exe
%run ./simdiv.exe

#include <stdio.h>
#include <stdint.h>
#include <assert.h>
    
int32_t simdiv(int32_t a) { 
    uint32_t eax = ((uint32_t)a >> 31) + a;
    __asm__("sar %0" : "=a"(eax) : "a"(eax));
    return eax;
}

int main() {
    assert(simdiv(1) == 0);
    assert(simdiv(5) == 2);
    assert(simdiv(-1) == 0);
    assert(simdiv(-5) == -2);
    printf("SUCCESS\n");
}

Run: `gcc -m64 -masm=intel -O3 simdiv.c -o simdiv.exe`

Run: `./simdiv.exe`

SUCCESS


##  <a name="hw"></a> Комментарии к дз


